<h1>5+15 EMA combined backtest</h1>
<hr>

In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import talib as ta
import yfinance as yf
import datetime

In [73]:
data1 = yf.download(tickers='ICICIBANK.NS', period='60d', interval='5m')
data1.to_csv("test.csv")

[*********************100%***********************]  1 of 1 completed


In [74]:
data2 = yf.download(tickers='ICICIBANK.NS', period='60d', interval='15m')
data2.to_csv("test1.csv")

[*********************100%***********************]  1 of 1 completed


In [75]:
data1 = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/test.csv')
data1['Datetime'] = pd.to_datetime(data1['Datetime'])
data1['5EMA'] = ta.EMA(data1['Close'],5)

data2 = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/test1.csv')
data2['Datetime'] = pd.to_datetime(data2['Datetime'])
data2['5EMA'] = ta.EMA(data2['Close'],5)

In [76]:
data1 = data1[data1['Datetime'].dt.month==11]
data2 = data2[data2['Datetime'].dt.month==11]

In [77]:
def enter_buy(data,j,sl,entry,target,arr,ana,count,slb_m):
    k = j+2
    while(k<len(data['Close'])):
        
        if data['Datetime'].dt.date.values[k] not in slb_m:
            slb_m[data['Datetime'].dt.date.values[k]] = 0
            
        if(data['Datetime'].dt.day.values[k]>data['Datetime'].dt.day.values[j]):
            return 
        if(data['Close'].values[k]<sl):
            change = (sl-entry)/entry
            arr.append(change)
            slb_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "-ve sl:{0} , entry:{1} , exit:{2} , target:{3} , entry-time{4} , exit-time:{5}".format(sl,entry,sl,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k])
            return k 
        if(data['Close'].values[k]>target):
            change = (target-entry)/entry
            arr.append(change)
            ana[count] = "+ve sl:{0} , entry:{1} , exit:{2} , target:{3} , entry-time{4} , exit-time:{5}".format(sl,entry,target,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k])
            return k
        if(data['5EMA'].values[k]-data['High'].values[k]>0.4):
            change = (data['Close'].values[k]-entry)/entry
            arr.append(change)
            ch_b = change>0
            if(change<0):
                slb_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "{6} sl:{0} , entry:{1} , exit:{2} , target:{3} , entry-time{4} , exit-time:{5}".format(sl,entry,data['Close'].values[k],target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],ch_b)
            return k-2
        k += 1

In [78]:
def trade_buy(data,i,arr,ana,count,slb_m):
    sl = 0.0
    sl_h = 0.0
    entry = 0.0
    target = 0.0
    j = i+1
    while(j<len(data['Close'])-1):
        if(data['High'].values[j]>data['5EMA'].values[j] and data['Low'].values[j]<data['5EMA'].values[j]):
            sl = data['Low'].values[j-1]
            entry = data['Open'].values[j+1] + ((data['Close'].values[j+1]-data['Open'].values[j+1])/2)
            target = entry + (3*(entry-sl))
            if(entry<sl):
                return j 
            return enter_buy(data,j,sl,entry,target,arr,ana,count,slb_m)
        j += 1

In [79]:
def enter_sell(data,j,sl,entry,target,arr,ana,count,sls_m):
    k = j+2
    while(k<len(data['Close'])):
        
        if data['Datetime'].dt.date.values[k] not in sls_m:
            sls_m[data['Datetime'].dt.date.values[k]] = 0
            
        if(data['Datetime'].dt.day.values[k]>data['Datetime'].dt.day.values[j]):
            return 
        
        if(data['Close'].values[k]>sl):
            change = (entry-sl)/entry
            arr.append(change)
            sls_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "-ve sl:{0} , entry:{1} , exit:{2} , target:{3} , entry-time{4} , exit-time:{5}".format(sl,entry,sl,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k])
            return k 
        if(data['Close'].values[k]<target):
            change = (entry-target)/entry
            arr.append(change)
            ana[count] = "+ve sl:{0} , entry:{1} , exit:{2} , target:{3} , entry-time{4} , exit-time:{5}".format(sl,entry,target,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k])
            return k
        if(data['Low'].values[k]-data['5EMA'].values[k]>0.4):
            change = (entry-data['Close'].values[k])/entry
            arr.append(change)
            ch_b = change>0
            if(change<0):
                sls_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "{6} sl:{0} , entry:{1} , exit:{2} , target:{3} , entry-time{4} , exit-time:{5}".format(sl,entry,data['Close'].values[k],target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],ch_b)
            return k-2
        k += 1

In [80]:
def trade_sell(data,i,arr,ana,count,sls_m):
    sl = 0.0
    sl_h = 0.0
    entry = 0.0
    target = 0.0
    j = i+1
    while(j<len(data['Close'])-1):
        if(data['High'].values[j]>data['5EMA'].values[j] and data['Low'].values[j]<data['5EMA'].values[j]):
            sl = data['High'].values[j-1]
            entry = data['Open'].values[j+1] + ((data['Close'].values[j+1]-data['Open'].values[j+1])/2)
            target = entry - (3*(sl-entry))
            if(entry>sl):
                return j 
            return enter_sell(data,j,sl,entry,target,arr,ana,count,sls_m)
        j += 1

In [81]:
def EMA(arr,data1,data2,ana,sls_m,slb_m):
    i1 = 0
    i2 = 0
    count = 0
    while(i1<len(data1['Close'])-1 or i2<len(data2['Close'])-1):
        
        if data1['Datetime'].dt.date.values[i1] not in sls_m:
            sls_m[data1['Datetime'].dt.date.values[i1]] = 0
            
        if(sls_m[data1['Datetime'].dt.date.values[i1]]>1):
            i1 += 1
            continue 
            
        if(data1['Low'].values[i1]>data1['5EMA'].values[i1]):
            count += 1
            a = trade_sell(data1,i1,arr,ana,count,sls_m)
            try:
                b = a-i1+1
                i1 += b
                continue
            except:
                c = 1+1
        
        i1 += 1
        
        if(i2 == len(data2['Close'])):
            continue
        
        if data2['Datetime'].dt.date.values[i2] not in slb_m:
            slb_m[data2['Datetime'].dt.date.values[i2]] = 0
            
        if(slb_m[data2['Datetime'].dt.date.values[i2]]>1):
            i2 += 1
            continue
        
        if(data2['High'].values[i2]<data2['5EMA'].values[i2]):
            count += 1
            x = trade_buy(data2,i2,arr,ana,count,slb_m)
            try:
                y = x-i2+1
                i2 += y
                continue
            except:
                z = 1+1
        
        i2 += 1
                
        

In [82]:
arr = []
ana = {}
sls_m = {}
slb_m = {}

EMA(arr,data1,data2,ana,sls_m,slb_m)

In [83]:
amount  = 10000
ini = amount
for i in arr:
    amount = (1+i)*amount
print(ini)
print(amount)
print(amount-ini)
print(((amount-ini)/ini)*100)

10000
10219.491764765884
219.491764765884
2.19491764765884


In [84]:
ana

{1: '+ve sl:914.1500244140624 , entry:912.5750122070312 , exit:907.8499755859378 , target:907.8499755859378 , entry-time09:30:00 , exit-time:12:10:00',
 3: '+ve sl:901.7000122070312 , entry:903.125 , exit:907.3999633789062 , target:907.3999633789062 , entry-time13:45:00 , exit-time:15:00:00',
 4: '-ve sl:905.6500244140624 , entry:906.5249938964844 , exit:905.6500244140624 , target:909.1499023437503 , entry-time11:30:00 , exit-time:12:15:00',
 7: 'False sl:908.9500122070312 , entry:906.6499938964844 , exit:908.9500122070312 , target:899.7499389648438 , entry-time15:00:00 , exit-time:15:10:00',
 10: 'False sl:897.0499877929688 , entry:906.2749938964844 , exit:905.8499755859376 , target:933.9500122070312 , entry-time09:45:00 , exit-time:14:15:00',
 12: '-ve sl:912.0 , entry:911.1499938964844 , exit:912.0 , target:908.5999755859375 , entry-time09:55:00 , exit-time:10:05:00',
 13: '+ve sl:913.5999755859376 , entry:911.7999877929688 , exit:906.4000244140622 , target:906.4000244140622 , entry